In [51]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json

pd.set_option('display.max_rows', 500)

RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html

John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19

REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

In [77]:
git_pull = subprocess.Popen( "/usr/bin/git pull" , 
                     cwd = os.path.dirname( 'C:/Users/saudf/ads_covid-19/data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : b'The system cannot find the path specified.\r\n'
out : b''


In [52]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [53]:
pd_raw

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,32022,32324,32672,32951,33190,33384,33594,33908,34194,34366
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,2662,2752,2819,2893,2964,3038,3106,3188,3278,3371
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,14657,15070,15500,15941,16404,16879,17348,17808,18242,18712
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,315,328,346,346,346,386,386,396,458,462
5,NaN,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,69,68,68,68,70,70,70,73,74,74
6,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,69941,72786,75376,77815,80447,83426,87030,90693,94060,97509
7,NaN,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,26658,27320,27900,28606,28936,29285,29820,30346,30903,31392
8,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,108,108,108,108,108,111,112,113,113,113
9,New South Wales,Australia,-33.868800,151.209300,0,0,0,0,3,4,...,3211,3405,3419,3429,3433,3440,3453,3467,3474,3478


## Webscrapping

In [54]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [55]:
soup = BeautifulSoup(page.content, 'html.parser')

In [56]:
html_table=soup.find('table')

In [57]:
all_rows=html_table.find_all('tr')

In [58]:
final_data_list=[]

In [59]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] 
    final_data_list.append(col_list)

In [60]:
pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,36.111,+70,208,"1,9",1.837
3,Bayern,49.350,+77,454,"3,5",2.613
4,Berlin,8.656,+18,168,"4,5",219
5,Branden­burg,3.462,0,15,"0,6",167
6,Bremen,1.691,+1,12,"1,8",55
7,Hamburg,5.231,0,11,"0,6",261
8,Hessen,11.206,+37,219,"3,5",514
9,Meck­lenburg-Vor­pommern,804,0,0,"0,0",20
10,Nieder­sachsen,13.823,+17,73,"0,9",642
11,Nord­rhein-West­falen,45.115,+141,918,"5,1",1.707


## REST API Calls

In [61]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [62]:
json_object=json.loads(data.content) 

In [63]:
type(json_object)

dict

In [64]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [65]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [66]:
pd.DataFrame(full_list)

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,3207,1594591200000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,110.711731,4.573731e+10,2.881496e+06,154
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,5231,1594591200000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,284.111431,2.089396e+09,4.188002e+05,261
2,3,03,Niedersachsen,Land,7982448,9,13806,1594591200000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,172.954462,1.299836e+11,4.008988e+06,642
3,4,04,Bremen,Freie Hansestadt,682986,5,1690,1594591200000,04,4132268b-54de-4327-ac1e-760e915112f1,247.442847,1.119157e+09,3.357177e+05,55
4,5,05,Nordrhein-Westfalen,Land,17932651,10,44974,1594591200000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,250.793929,8.782936e+10,2.648673e+06,1705
5,6,06,Hessen,Land,6265809,7,11169,1594591200000,06,93277ac4-e8fc-48c7-8940-028dc2ed66af,178.253119,5.235913e+10,2.148244e+06,514
6,7,07,Rheinland-Pfalz,Land,4084844,11,7174,1594591200000,07,e9b4296f-9be2-4e53-9a58-ccf1396cb03d,175.624822,4.783877e+10,1.774430e+06,236
7,8,08,Baden-Württemberg,Land,11069533,1,36041,1594591200000,08,80394ddf-c6a4-4a6e-be8e-0259a81b22a9,325.587358,8.151732e+10,2.544320e+06,1837
8,9,09,Bayern,Freistaat,13076721,2,49273,1594591200000,09,1ff920f4-62cd-4a4f-b8c9-f042f2a3e00a,376.799352,1.634855e+11,3.898618e+06,2612
9,10,10,Saarland,Land,990509,12,2812,1594591200000,10,e3396a6f-8a30-4fdf-8df7-def77dd38bea,283.894442,6.060692e+09,5.626789e+05,174


## API access via REST service (US Data)

In [67]:
url_endpoint='https://api.smartable.ai/coronavirus/stats/US'

headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key': '28ee4219700f48718be78b057beb7eb4',
}

response = requests.get(url_endpoint, headers=headers)

In [68]:
print(response)

<Response [200]>


In [50]:
US_dict=json.loads(response.content)
with open('C:/Users/saudf/ads_covid-19/data/raw/US_data.txt', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)